In [3]:
!pip install boto3

   ---------------------------------------- 0.0/139.6 kB ? eta -:--:--
   -------- ------------------------------- 30.7/139.6 kB ? eta -:--:--
   -------- ------------------------------- 30.7/139.6 kB ? eta -:--:--
   ----------------- --------------------- 61.4/139.6 kB 328.2 kB/s eta 0:00:01
   ------------------------------ ------- 112.6/139.6 kB 504.4 kB/s eta 0:00:01
   ------------------------------ ------- 112.6/139.6 kB 504.4 kB/s eta 0:00:01
   ------------------------------------ - 133.1/139.6 kB 437.3 kB/s eta 0:00:01
   -------------------------------------- 139.6/139.6 kB 394.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/13.4 MB ? eta -:--:--
    --------------------------------------- 0.2/13.4 MB 3.5 MB/s eta 0:00:04
    --------------------------------------- 0.2/13.4 MB 3.5 MB/s eta 0:00:04
    --------------------------------------- 0.3/13.4 MB 1.9 MB/s eta 0:00:07
   - -------------------------------------- 0.4/13.4 MB 2.1 MB/s eta 0:00:07
   - --

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.37.14 which is incompatible.


In [8]:
import boto3
import json
import time
import zipfile
import os

# Configuration
region = 'us-east-1'
bucket_name = f'energy-monitoring-data-{int(time.time())}'  # Unique bucket name

# AWS clients with explicit region
iot = boto3.client('iot', region_name=region)
lambda_client = boto3.client('lambda', region_name=region)
firehose = boto3.client('firehose', region_name=region)
s3 = boto3.client('s3', region_name=region)
glue = boto3.client('glue', region_name=region)
iam = boto3.client('iam', region_name=region)
kms = boto3.client('kms', region_name=region)

print("All clients initialized successfully.")

All clients initialized successfully.


In [11]:
try:
    # Create KMS key
    kms_response = kms.create_key(
        Description='KMS key for energy monitoring',
        KeyUsage='ENCRYPT_DECRYPT',
        MultiRegion=False
    )
    kms_key_id = kms_response['KeyMetadata']['KeyId']
    kms_key_arn = kms_response['KeyMetadata']['Arn']
    print(f"KMS Key created: {kms_key_arn}")
except Exception as e:
    print(f"Error creating KMS key: {str(e)}")

Error creating KMS key: Unable to locate credentials


In [13]:
try:
    # Create S3 bucket
    s3.create_bucket(Bucket=bucket_name)
    
    # Enable server-side encryption with KMS
    s3.put_bucket_encryption(
        Bucket=bucket_name,
        ServerSideEncryptionConfiguration={
            'Rules': [{
                'ApplyServerSideEncryptionByDefault': {
                    'SSEAlgorithm': 'aws:kms',
                    'KMSMasterKeyID': kms_key_id
                }
            }]
        }
    )
    print(f"S3 Bucket created and encrypted: {bucket_name}")
except Exception as e:
    print(f"Error creating S3 bucket: {str(e)}")

Error creating S3 bucket: Unable to locate credentials


In [ ]:
try:
    # Lambda execution role
    assume_role_policy = {
        "Version": "2012-10-17",
        "Statement": [{"Effect": "Allow", "Principal": {"Service": "lambda.amazonaws.com"}, "Action": "sts:AssumeRole"}]
    }
    lambda_role = iam.create_role(
        RoleName='LambdaEnergyMonitorRole',
        AssumeRolePolicyDocument=json.dumps(assume_role_policy)
    )
    iam.attach_role_policy(
        RoleName=lambda_role['Role']['RoleName'],
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    iam.put_role_policy(
        RoleName=lambda_role['Role']['RoleName'],
        PolicyName='FirehoseAccess',
        PolicyDocument=json.dumps({
            "Version": "2012-10-17",
            "Statement": [{"Effect": "Allow", "Action": ["firehose:PutRecord"], "Resource": "*"}]
        })
    )
    
    # Kinesis Firehose role
    firehose_role = iam.create_role(
        RoleName='FirehoseEnergyRole',
        AssumeRolePolicyDocument=json.dumps({
            "Version": "2012-10-17",
            "Statement": [{"Effect": "Allow", "Principal": {"Service": "firehose.amazonaws.com"}, "Action": "sts:AssumeRole"}]
        })
    )
    iam.put_role_policy(
        RoleName=firehose_role['Role']['RoleName'],
        PolicyName='S3KMSAccess',
        PolicyDocument=json.dumps({
            "Version": "2012-10-17",
            "Statement": [
                {"Effect": "Allow", "Action": ["s3:PutObject", "s3:PutObjectAcl"], "Resource": f"arn:aws:s3:::{bucket_name}/*"},
                {"Effect": "Allow", "Action": ["kms:Decrypt"], "Resource": kms_key_arn}
            ]
        })
    )
    
    # Glue role
    glue_role = iam.create_role(
        RoleName='GlueEnergyRole',
        AssumeRolePolicyDocument=json.dumps({
            "Version": "2012-10-17",
            "Statement": [{"Effect": "Allow", "Principal": {"Service": "glue.amazonaws.com"}, "Action": "sts:AssumeRole"}]
        })
    )
    iam.attach_role_policy(
        RoleName=glue_role['Role']['RoleName'],
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSGlueServiceRole'
    )
    iam.put_role_policy(
        RoleName=glue_role['Role']['RoleName'],
        PolicyName='S3Access',
        PolicyDocument=json.dumps({
            "Version": "2012-10-17",
            "Statement": [{"Effect": "Allow", "Action": ["s3:GetObject", "s3:ListBucket"], "Resource": [f"arn:aws:s3:::{bucket_name}", f"arn:aws:s3:::{bucket_name}/*"]}]
        })
    )
    
    print("IAM roles created successfully.")
except Exception as e:
    print(f"Error creating IAM roles: {str(e)}")

Error creating IAM roles: Unable to locate credentials


In [17]:
try:
    # Write Lambda code to a file
    lambda_code = '''
import json
import boto3

firehose = boto3.client('firehose', region_name='us-east-1')

def lambda_handler(event, context):
    energy_data = {
        "timestamp": event.get("timestamp"),
        "kwh": event.get("kwh")
    }
    firehose.put_record(
        DeliveryStreamName='energy-firehose-stream',
        Record={'Data': json.dumps(energy_data).encode('utf-8')}
    )
    return {"status": "success"}
'''
    with open('lambda_function.py', 'w') as f:
        f.write(lambda_code)

    # Zip the Lambda code
    with zipfile.ZipFile('lambda_function.zip', 'w') as z:
        z.write('lambda_function.py')

    # Create Lambda function (wait briefly for IAM role to propagate)
    time.sleep(10)  # Avoid IAM role propagation delay
    lambda_response = lambda_client.create_function(
        FunctionName='energyProcessor',
        Runtime='python3.9',
        Role=lambda_role['Role']['Arn'],
        Handler='lambda_function.lambda_handler',
        Code={'ZipFile': open('lambda_function.zip', 'rb').read()},
        Timeout=10
    )
    print(f"Lambda created: {lambda_response['FunctionArn']}")

    # Clean up local files
    os.remove('lambda_function.py')
    os.remove('lambda_function.zip')
except Exception as e:
    print(f"Error creating Lambda function: {str(e)}")

Error creating Lambda function: name 'lambda_role' is not defined


In [19]:
try:
    # Create IoT topic rule to trigger Lambda
    iot.create_topic_rule(
        ruleName='energy_to_lambda',
        topicRulePayload={
            "sql": "SELECT * FROM 'energy/monitor'",
            "actions": [{"lambda": {"functionArn": lambda_response['FunctionArn']}}],
            "ruleDisabled": False
        }
    )
    print("IoT Rule created successfully.")
except Exception as e:
    print(f"Error creating IoT rule: {str(e)}")

Error creating IoT rule: name 'lambda_response' is not defined


In [21]:
try:
    # Create Firehose delivery stream
    firehose.create_delivery_stream(
        DeliveryStreamName='energy-firehose-stream',
        S3DestinationConfiguration={
            'RoleARN': firehose_role['Role']['Arn'],
            'BucketARN': f'arn:aws:s3:::{bucket_name}',
            'BufferingHints': {'SizeInMBs': 5, 'IntervalInSeconds': 60},  # Batching
            'EncryptionConfiguration': {'KMSEncryptionConfig': {'AWSKMSKeyARN': kms_key_arn}}
        }
    )
    # Wait for Firehose to become active
    time.sleep(20)
    print("Kinesis Firehose created successfully.")
except Exception as e:
    print(f"Error creating Firehose stream: {str(e)}")

Error creating Firehose stream: name 'firehose_role' is not defined


In [23]:
try:
    # Simulate IoT data and invoke Lambda
    test_event = {"timestamp": "2025-03-18T10:00:00Z", "kwh": 1.5}
    response = lambda_client.invoke(
        FunctionName='energyProcessor',
        InvocationType='RequestResponse',
        Payload=json.dumps(test_event)
    )
    print("Lambda Response:", json.loads(response['Payload'].read()))
except Exception as e:
    print(f"Error testing Lambda: {str(e)}")

Error testing Lambda: Unable to locate credentials
